<a href="https://colab.research.google.com/github/paulmachau/K-means-clustering-/blob/main/Phoenix_KE_Analytics_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#loading the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#loading the dataset
df = pd.read_csv('/content/Synthetic Agent Data.csv')
df.head()

,Unnamed: 0,Agent 0,Agent 1,Agent 2,Agent 3,Agent 4,Agent 5,Agent 6,Agent 7,Agent 8,...,Agent 1190,Agent 1191,Agent 1192,Agent 1193,Agent 1194,Agent 1195,Agent 1196,Agent 1197,Agent 1198,Agent 1199
0,0,1,2,1,0,1,1,0,0,0,...,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0
1,1,9,10,8,2,15,7,0,2,2,...,0.0,9.0,8.0,0.0,14.0,0.0,1.0,0.0,7.0,1.0
2,2,9,7,11,3,10,7,0,1,3,...,3.0,9.0,10.0,0.0,9.0,0.0,2.0,1.0,12.0,0.0
3,3,11,14,13,0,8,9,0,0,3,...,1.0,14.0,11.0,1.0,7.0,2.0,3.0,1.0,10.0,2.0
4,4,11,13,7,1,7,8,1,0,4,...,2.0,14.0,8.0,0.0,7.0,1.0,2.0,1.0,15.0,1.0


In [3]:
#defining the data set
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Columns: 1201 entries, Unnamed: 0 to Agent 1199
dtypes: float64(917), int64(283), object(1)
memory usage: 797.7+ KB


In [4]:
# checking for missing values

df.isnull().sum()


Unnamed: 0    0
Agent 0       0
Agent 1       0
Agent 2       0
Agent 3       0
             ..
Agent 1195    1
Agent 1196    1
Agent 1197    1
Agent 1198    1
Agent 1199    1
Length: 1201, dtype: int64

In [5]:
#removing missing values

df = df.dropna()


In [6]:
#droping the first column

df = df.iloc[: , 1:]
df.head()


,Agent 0,Agent 1,Agent 2,Agent 3,Agent 4,Agent 5,Agent 6,Agent 7,Agent 8,Agent 9,...,Agent 1190,Agent 1191,Agent 1192,Agent 1193,Agent 1194,Agent 1195,Agent 1196,Agent 1197,Agent 1198,Agent 1199
0,1,2,1,0,1,1,0,0,0,2,...,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0
1,9,10,8,2,15,7,0,2,2,11,...,0.0,9.0,8.0,0.0,14.0,0.0,1.0,0.0,7.0,1.0
2,9,7,11,3,10,7,0,1,3,11,...,3.0,9.0,10.0,0.0,9.0,0.0,2.0,1.0,12.0,0.0
3,11,14,13,0,8,9,0,0,3,14,...,1.0,14.0,11.0,1.0,7.0,2.0,3.0,1.0,10.0,2.0
4,11,13,7,1,7,8,1,0,4,13,...,2.0,14.0,8.0,0.0,7.0,1.0,2.0,1.0,15.0,1.0


In [7]:
# Transpose the dataframe so that each row represents an agent
df_transposed = df.transpose()

# Rename columns to represent days
df_transposed.columns = [f'Day_{i}' for i in range(df_transposed.shape[1])]


In [8]:
#print the new data set

df_transposed.head()


,Day_0,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,...,Day_74,Day_75,Day_76,Day_77,Day_78,Day_79,Day_80,Day_81,Day_82,Day_83
Agent 0,1.0,9.0,9.0,11.0,11.0,8.0,1.0,2.0,12.0,7.0,...,7.0,7.0,1.0,2.0,7.0,8.0,7.0,11.0,9.0,2.0
Agent 1,2.0,10.0,7.0,14.0,13.0,11.0,2.0,2.0,9.0,7.0,...,14.0,8.0,1.0,2.0,7.0,8.0,11.0,7.0,9.0,2.0
Agent 2,1.0,8.0,11.0,13.0,7.0,14.0,2.0,2.0,15.0,13.0,...,7.0,9.0,2.0,1.0,9.0,7.0,7.0,9.0,11.0,2.0
Agent 3,0.0,2.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
Agent 4,1.0,15.0,10.0,8.0,7.0,12.0,2.0,2.0,8.0,8.0,...,7.0,10.0,2.0,2.0,9.0,10.0,7.0,13.0,7.0,2.0


In [9]:

# Calculate mean, median, max, and min for each agent
agent_stats = pd.DataFrame()
agent_stats['Mean'] = df_transposed.mean(axis=1)
agent_stats['Median'] = df_transposed.median(axis=1)
agent_stats['Max'] = df_transposed.max(axis=1)
agent_stats['Min'] = df_transposed.min(axis=1)

print(agent_stats)


                Mean  Median   Max  Min
Agent 0     6.940476     7.0  14.0  1.0
Agent 1     6.702381     7.0  15.0  1.0
Agent 2     7.000000     7.5  15.0  1.0
Agent 3     0.535714     0.0   3.0  0.0
Agent 4     7.297619     8.0  15.0  1.0
...              ...     ...   ...  ...
Agent 1195  0.309524     0.0   2.0  0.0
Agent 1196  2.083333     2.0   6.0  0.0
Agent 1197  1.714286     1.0   8.0  0.0
Agent 1198  7.285714     7.0  15.0  1.0
Agent 1199  0.595238     0.0   3.0  0.0

[1200 rows x 4 columns]


In [10]:
# Calculate overall statistics
overall_stats = pd.DataFrame()
overall_stats['Mean'] = [df_transposed.values.mean()]
overall_stats['Median'] = [np.median(df_transposed.values)]
overall_stats['Max'] = [df_transposed.values.max()]
overall_stats['Min'] = [df_transposed.values.min()]

# Display the overall statistics
print("\nOverall statistics:\n", overall_stats)



Overall statistics:
        Mean  Median   Max  Min
0  3.542857     1.0  24.0  0.0


**Inference**

* Overall Mean: The average number of tasks completed across all agents and all days is approximately 3.54 tasks per day.
* Overall Median: The median number of tasks completed is 5, indicating that on most days, agents completed around 1 tasks.

* Overall Max: The maximum number of tasks completed in a single day by any agent is 24.
* Overall Min: The minimum number of tasks completed in a single day is 0, indicating some days with no task completions.

From the summury statistics it shows a high variation between the max and Min task done in a day  which has resulted in a low frequency of number of tasks completed in a day indicated by the median



In [11]:
#the standard deviation of their number of tasks completed

agent_stats['Std Dev'] = df_transposed.std(axis=1)
print(agent_stats)


                Mean  Median   Max  Min   Std Dev
Agent 0     6.940476     7.0  14.0  1.0  3.816108
Agent 1     6.702381     7.0  15.0  1.0  3.867628
Agent 2     7.000000     7.5  15.0  1.0  4.032996
Agent 3     0.535714     0.0   3.0  0.0  0.870486
Agent 4     7.297619     8.0  15.0  1.0  4.236331
...              ...     ...   ...  ...       ...
Agent 1195  0.309524     0.0   2.0  0.0  0.559177
Agent 1196  2.083333     2.0   6.0  0.0  1.716036
Agent 1197  1.714286     1.0   8.0  0.0  1.923673
Agent 1198  7.285714     7.0  15.0  1.0  4.249937
Agent 1199  0.595238     0.0   3.0  0.0  0.879746

[1200 rows x 5 columns]


In [12]:
# prompt: the standard deviation for overall agents combined

overall_std = df_transposed.values.std()
print("\nOverall standard deviation:", overall_std)



Overall standard deviation: 4.230781543835055
